This notebook takes the numveh (base year vehicle numveh data) as an input and produces the newveh (base year *new* vehicles) as an output. To do this it uses the GIZ data.

In [23]:
import pandas as pd

In [24]:
# read kenya TED -- new registrations per year
newreg = pd.read_excel(f'./../TED/Kenya_TED.xlsx', sheet_name='New-registrations-by-year')
newreg

,Vehicle type,2015,2016,2017,2018,2019
0,Motorcycles,139420,123539,191601,195253,217425
1,Cars,68489,58613,66698,74683,82483
2,LCVs,13878,12722,9866,11220,10189
3,HGVs,13785,9632,7460,6514,6518
4,Buses and minibuses,4781,4096,3563,5659,5651
5,Trailers,3905,2829,1953,2083,1639
6,Other motor vehicles,2923,2284,1531,1877,3271
7,Total,247181,213715,282672,297289,327176


In [25]:
# read UK data on new vehicles other types - ships, planes, trains
newvehuk = pd.read_excel('./../sample_data/NewVeh95_UK.xlsx')
newvehuk

,CountryID,TransTypeID,VehTypeID,MassCatID,NumNewVeh
0,9,1,2,4,1.545915e+05
1,9,1,3,1,1.022889e+06
2,9,1,3,2,1.034924e+06
3,9,1,3,3,4.847108e+05
4,9,1,4,11,6.282496e+03
5,9,1,4,12,4.530616e+03
6,9,1,4,13,2.671586e+02
7,9,1,6,31,3.150600e+01
8,9,1,6,32,1.730470e+02
9,9,1,6,33,1.820150e+01


In [26]:
# defined assumed proportion of new kenya fleet/new uk fleet
#train_proportion, ship_proportion, plane_proportion = 0.1, 0.1, 0.1
uk_kenya_translation = {6: 0.1, 7: 0.1, 8: 0.1}

In [27]:
# make a vehtype dictionary
vehtype_dict = {2: ['Motorcycles'], 3: ['Cars'], 4: ['Buses and minibuses'], 
               5: ['LCVs', 'HGVs']}

In [28]:
# read aemda output data - vehicle numveh
numveh = pd.read_csv(f'./../out_data/processed/numveh95.csv')
#numveh = pd.read_csv(f'./out_data/processed/afema_bodge/numveh95_afemabodge.csv')
numveh


,CountryID,TechID,Age,TransTypeID,VehTypeID,MassCatID,EngineID,FuelID,NumVeh
0,9,10.0,0,1.0,2.0,4.0,10.0,1.0,61274.572000
1,9,10.0,1,1.0,2.0,4.0,10.0,1.0,88740.372000
2,9,10.0,2,1.0,2.0,4.0,10.0,1.0,34836.824000
3,9,10.0,3,1.0,2.0,4.0,10.0,1.0,43911.816000
4,9,10.0,4,1.0,2.0,4.0,10.0,1.0,200302.196000
...,...,...,...,...,...,...,...,...,...
4138,9,10690.0,14,2.0,5.0,23.0,15.0,2.0,207.771875
4139,9,10690.0,15,2.0,5.0,23.0,15.0,2.0,69.257292
4140,9,10690.0,16,2.0,5.0,23.0,15.0,2.0,138.514583
4141,9,10690.0,21,2.0,5.0,23.0,15.0,2.0,69.257292


In [29]:
newveh = pd.DataFrame(columns=['CountryID','TransTypeID', 'VehTypeID', 'MassCatID', 'NumNewVeh'])
newveh

,CountryID,TransTypeID,VehTypeID,MassCatID,NumNewVeh


In [30]:
base_year = 2015
row_count = 0

for transtype in numveh.TransTypeID.unique():
    
    for vehtype in numveh.VehTypeID.unique():
        
        for masscat in numveh.MassCatID.unique():
            
            if not numveh[(numveh.TransTypeID == transtype) & (numveh.VehTypeID == vehtype) & (numveh.MassCatID == masscat)].empty:
                
                # return total numveh of this config
                total_numveh = numveh[(numveh.TransTypeID == transtype) & (numveh.VehTypeID == vehtype) & (numveh.MassCatID == masscat)].NumVeh.sum()
                                
                # return new registrations of this vehicle type in the base year
                if vehtype in vehtype_dict:  # if it's a road vehicle
                    
                    new_registrations = newreg[(newreg['Vehicle type'].isin(vehtype_dict[vehtype]))][base_year].sum()
                    
                    # return total numveh of this vehicle type
                    total_vehtype = numveh[numveh.VehTypeID == vehtype].NumVeh.sum()
                    
                    # calculate new registrations as a *proportion* of the total of this vehicle type
                    newveh_proportion = new_registrations / total_vehtype
                    
                    numnewveh = newveh_proportion * total_numveh
                    
                else:
                    # in this case we take from the uk data and multiply by a fixed proportion
                    new_registrations = newvehuk[(newvehuk.TransTypeID == transtype) & (newvehuk.VehTypeID == vehtype) & (newvehuk.MassCatID == masscat)].NumNewVeh.sum()
                
                    numnewveh = new_registrations * uk_kenya_translation[vehtype]
                
                newveh.at[row_count, 'CountryID'] = 9
                newveh.at[row_count, 'TransTypeID'] = transtype
                newveh.at[row_count, 'VehTypeID'] = vehtype
                newveh.at[row_count, 'MassCatID'] = masscat
                newveh.at[row_count, 'NumNewVeh'] = numnewveh

                row_count +=1
                
newveh


,CountryID,TransTypeID,VehTypeID,MassCatID,NumNewVeh
0,9,1.0,2.0,4.0,139420.0
1,9,1.0,3.0,1.0,52683.064415
2,9,1.0,3.0,2.0,13777.788693
3,9,1.0,3.0,3.0,2028.146892
4,9,1.0,4.0,11.0,3033.662976
5,9,1.0,4.0,12.0,1455.119855
6,9,1.0,4.0,13.0,292.217169
7,9,1.0,6.0,31.0,3.1506
8,9,1.0,6.0,32.0,17.3047
9,9,1.0,6.0,33.0,1.82015


In [31]:
newveh.to_csv('./../out_data/processed/newveh95.csv', index=False)


In [32]:
newveh[newveh.VehTypeID==6].NumNewVeh.sum()

#VALIDATED IT IS CORRECT ACCORDING TO THE PROPORTION ABOVE

24.75345

In [33]:
newveh[newveh.duplicated()]

,CountryID,TransTypeID,VehTypeID,MassCatID,NumNewVeh
